In [1]:
# !pip install loguru
# !pip install nltk

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import warnings
# Download necessary NLTK corpora
# nltk.download('stopwords')

In [3]:
warnings.filterwarnings("ignore")

In [4]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("finnlp_binary_undersampling_experiment_1.log", rotation="1024 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-03-25 23:54:15.035 | INFO     | __main__:<module>:5 - 

####################################### NEW EXECUTION #######################################


2024-03-25 23:54:15.036 | INFO     | __main__:<module>:6 - Processors (cores) available: 10


In [5]:
bbrc = pd.read_csv("BBRC_Brazilian_Banking_Regulation_Corpora.csv" ,encoding="utf-8", low_memory=False, sep=";")

In [6]:
bbrc.head(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
0,1,usi,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984
1,1,usi,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973
2,1,usi,2020-12-22,2130,36554,2020-12-22,BACEN,Divulga comunicado do Grupo de Ação Financeira...,120,16.0,18.0,Comunicamos que o Grupo de Ação Financeira con...,792,76.0,127.0,COMUNICADO,771452
3,1,usi,2020-12-23,2131,4880,2020-12-23,BACEN,Dispõe sobre o horário de atendimento ao públi...,222,32.0,34.0,"O Banco Central do Brasil, na forma do art. 9º...",4964,365.0,875.0,RESOLUÇÃO CMN,772051
4,1,usi,2020-12-23,2132,1125/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,165,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3382,304.0,492.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771966
5,1,usi,2020-12-23,2133,1120/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,169,19.0,24.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3201,287.0,480.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771967
6,1,usi,2020-12-23,2134,1122/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,169,19.0,24.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3215,289.0,482.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771968
7,1,usi,2020-12-23,2135,1123/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,156,18.0,22.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2990,277.0,430.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771969
8,1,usi,2020-12-24,2136,36565,2020-12-24,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1574,149.0,280.0,COMUNICADO,772675
9,1,usi,2020-12-24,2137,1061/2020,2020-12-23,BACEN,Caracterizado fornecimento intempestivo de in...,467,57.0,88.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,859,97.0,146.0,PROCESSO ADMINISTRATIVO SANCIONADOR,772640


In [7]:
bbrc.tail(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
61844,0,uni,2020-07-28,112246,1654,2020-06-19,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Inclui o art. 2º-B e altera os arts. 3º e 5º d...,284,36.0,41.0,"A CONTROLADORA-GERAL DO MUNICÍPIO, no uso das ...",4359,363.0,706.0,RESOLUÇÃO,707723
61845,0,uni,2020-07-28,112247,3163,2020-06-18,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre operações de fiscalização das\r\n...,69,9.0,9.0,"A SECRETÁRIA MUNICIPAL DE FAZENDA, no uso das ...",2668,266.0,456.0,RESOLUÇÃO,707691
61846,0,uni,2020-07-28,112248,47533,2020-06-11,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"Altera o Decreto Rio nº 47.282, de 21 de março...",213,29.0,31.0,"O PREFEITO DA CIDADE DO RIO DE JANEIRO, no uso...",4825,270.0,476.0,DECRETO,705560
61847,0,uni,2020-07-28,112249,57,2020-06-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"/Dispõe sobre a prorrogação da licença médica,...",59,9.0,9.0,O SUBSECRETÁRIO DE SERVIÇOS COMPARTILHADOS DA ...,1370,155.0,220.0,PORTARIA,705564
61848,0,uni,2020-07-28,112250,1002,2020-06-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Prorroga o prazo de validade das certidões de\...,86,14.0,14.0,O PROCURADOR-GERAL DO MUNICÍPIO DO RIO DE JANE...,2268,188.0,370.0,RESOLUÇÃO,706277
61849,0,uni,2020-07-28,112251,157,2020-06-05,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Altera a composição da Comissão de\r\nPrograma...,104,15.0,16.0,A PRESIDENTE DA COMISSÃO DE PROGRAMAÇÃO E CONT...,642,86.0,111.0,DELIBERAÇÃO,703886
61850,0,dicre,2021-11-09,57669,7.107,2021-11-04,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre a margem consignável dos servidor...,243,28.0,31.0,Legislação - Lei Ordinária Lei nº 7107/2021 Da...,1963,207.0,307.0,LEI,877367
61851,0,dicre,2021-10-28,57670,49695,2021-10-27,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Cria a Área de Proteção Ambiental do Sertão Ca...,52,9.0,9.0,"DECRETO RIO Nº 49695, DE 27 DE OUTUBRO DE 2021...",18456,879.0,2932.0,DECRETO,873815
61852,0,dicre,2021-10-28,57671,49696,2021-10-27,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Estabelece o zoneamento da área de proteção am...,159,24.0,27.0,"DECRETO RIO Nº 49696, DE 27 DE OUTUBRO DE 2021...",16436,872.0,2721.0,DECRETO,873814
61853,0,uac,2021-04-26,50249,48768,2021-04-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre as competências da Empresa Munici...,81,12.0,12.0,DECRETO RIO Nº 48768 DE 23 DE ABRIL DE 2021 Di...,65853,1963.0,9767.0,DECRETO,809105


In [8]:
data_irrelevant = bbrc[bbrc['class'] == 0]

In [9]:
data_irrelevant.head(10)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
7846,0,usi,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080
7847,0,usi,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131
7848,0,usi,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151
7849,0,usi,2020-08-31,2172,705/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2930,275.0,432.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729089
7850,0,usi,2020-08-31,2173,703/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2930,277.0,432.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729091
7851,0,usi,2020-08-31,2174,701/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2920,275.0,429.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729093
7852,0,usi,2020-08-31,2175,700/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2871,271.0,424.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729094
7853,0,usi,2020-08-31,2176,704/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2875,271.0,426.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729090
7854,0,usi,2020-08-31,2177,702/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2913,276.0,430.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729092
7855,0,usi,2020-08-31,2178,699/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2866,271.0,425.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729095


In [10]:
data_irrelevant.tail(10)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
61854,0,uac,2021-05-07,50250,48845,2021-05-06,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre as medidas emergenciais restritiv...,165,20.0,22.0,"DECRETO RIO Nº 48845, DE 6 DE MAIO DE 2021. Di...",8477,631.0,1339.0,DECRETO,813206
61855,0,uac,2021-07-27,74425,7.003,2021-07-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre a venda de produtos\r\nhortifruti...,142,14.0,15.0,"LEI Nº 7.003, DE 23 DE JULHO DE 2021 Dispõe so...",1586,165.0,244.0,LEI,841036
61856,0,uac,2021-07-27,74426,7.004,2021-07-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre o sistema de cobranças dos pedági...,494,58.0,63.0,"LEI Nº 7.004, DE 23 DE JULHO DE 2021 Dispõe so...",1153,143.0,185.0,LEI,841037
61857,0,uac,2021-07-27,74427,7003,2021-07-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre a venda de produtos hortifrutigra...,104,13.0,14.0,Legislação - Lei Ordinária Lei nº 7003/2021 Da...,2144,210.0,331.0,LEI,841060
61858,0,uac,2021-07-27,74428,7004,2021-07-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre o sistema de cobranças dos pedági...,116,17.0,19.0,Legislação - Lei Ordinária Lei nº 7004/2021 Da...,2143,227.0,329.0,LEI,841061
61859,0,uac,2020-09-08,108108,4525,2020-09-04,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Altera o item 4 do Anexo da Resolução SMS nº\r...,326,38.0,46.0,"A SECRETÁRIA MUNICIPAL DE SAÚDE, no uso das at...",20171,1213.0,3118.0,RESOLUÇÃO,730929
61860,0,uac,2020-08-24,108109,05,2020-08-19,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Altera e consolida o Regulamento do Fundo Fixo...,91,15.0,15.0,O DIRETOR-PRESIDENTE DA DISTRIBUIDORA DE FILME...,9514,671.0,1489.0,PORTARIA,727088
61861,0,uac,2020-08-12,108110,280,2020-08-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"INSTITUI O PLANO DE CLASSIFICAÇÃO, A TABELA DE...",141,17.0,19.0,O PRESIDENTE DA EMPRESA MUNICIPAL DE INFORMÁTI...,83300,2995.0,12054.0,PORTARIA,723644
61862,0,uac,2020-07-28,108111,557,2020-06-08,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre os programas de autocontrole (PAC...,268,31.0,33.0,"A SUBSECRETÁRIA DE VIGILÂNCIA, FISCALIZAÇÃO SA...",56226,2793.0,8432.0,PORTARIA,708087
61863,0,uac,2020-07-28,108112,560,2020-06-08,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre os requisitos técnicos relativos ...,232,25.0,28.0,"A SUBSECRETÁRIA DE VIGILÂNCIA, FISCALIZAÇÃO SA...",64411,2626.0,9348.0,PORTARIA,708100


In [11]:
data_relevant = bbrc[bbrc['class'] == 1]

In [12]:
data_relevant.count()

class                   7846
department              7846
entry_date              7846
general_id              7846
normative_identifier    7846
publication_date        7846
regulatory_authority    7846
subject                 7846
subject_length          7846
subject_unique_words    7846
subject_words           7846
text                    7846
text_length             7846
text_unique_words       7846
text_words              7846
type                    7846
unique_document_id      7846
dtype: int64

In [13]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
dtype: int64

In [14]:
data_irrelevant.count()

class                   54018
department              54018
entry_date              54018
general_id              54018
normative_identifier    54018
publication_date        54018
regulatory_authority    54018
subject                 54018
subject_length          54018
subject_unique_words    54018
subject_words           54018
text                    54018
text_length             54018
text_unique_words       54018
text_words              54018
type                    54018
unique_document_id      54018
dtype: int64

In [15]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
dtype: int64

In [16]:
data_relevant['department'].unique()

array(['usi', 'cib', 'bb_asset', 'dimep', 'coger gesub', 'disem',
       'upb/mercap', 'tesou', 'diope gefid', 'diope', 'dirag', 'coger',
       'digov', 'uci', 'bb seguros', 'diris', 'ucf', 'uge', 'dined',
       'uni', 'dicre', 'dipes', 'ditec', 'uac', 'coger getri'],
      dtype=object)

In [17]:
len(list(data_relevant['department'].unique()))

25

In [18]:
stop_words = set(stopwords.words('portuguese'))
df_stopwords = pd.DataFrame(list(stop_words), columns =['words'])
df_stopwords.head(10)

,words
0,entre
1,fui
2,para
3,houvéramos
4,muito
5,na
6,até
7,minha
8,nos
9,eu


In [19]:
df_stopwords.tail(10)

,words
197,não
198,suas
199,seja
200,somos
201,nós
202,eram
203,houvessem
204,numa
205,de
206,estivemos


In [20]:
def remove_special_characters(df,column):
    df.loc[:,column] = df[column].str.lower()
    
    df.loc[:,column] = df[column].str.replace('á','a')
    df.loc[:,column] = df[column].str.replace('é','e')
    df.loc[:,column] = df[column].str.replace('í','i')
    df.loc[:,column] = df[column].str.replace('ó','o')
    df.loc[:,column] = df[column].str.replace('ú','u')

    df.loc[:,column] = df[column].str.replace('â','a')
    df.loc[:,column] = df[column].str.replace('ê','e')
    df.loc[:,column] = df[column].str.replace('î','i')
    df.loc[:,column] = df[column].str.replace('ô','o')
    df.loc[:,column] = df[column].str.replace('û','u')

    df.loc[:,column] = df[column].str.replace('à','a')
    df.loc[:,column] = df[column].str.replace('è','e')
    df.loc[:,column] = df[column].str.replace('ì','i')
    df.loc[:,column] = df[column].str.replace('ò','o')
    df.loc[:,column] = df[column].str.replace('ù','u')

    df.loc[:,column] = df[column].str.replace('ä','a')
    df.loc[:,column] = df[column].str.replace('ë','e')
    df.loc[:,column] = df[column].str.replace('ï','i')
    df.loc[:,column] = df[column].str.replace('ö','o')
    df.loc[:,column] = df[column].str.replace('ü','u')

    df.loc[:,column] = df[column].str.replace('ã','a')
    df.loc[:,column] = df[column].str.replace('ẽ','e')
    df.loc[:,column] = df[column].str.replace('ĩ','i')
    df.loc[:,column] = df[column].str.replace('õ','o')
    df.loc[:,column] = df[column].str.replace('ũ','u')

    df.loc[:,column] = df[column].str.replace('ç','c')
    
    return df

In [21]:
df_stopwords = remove_special_characters(df_stopwords,'words')

In [22]:
df_stopwords.head(10)

,words
0,entre
1,fui
2,para
3,houveramos
4,muito
5,na
6,ate
7,minha
8,nos
9,eu


In [23]:
df_stopwords.tail(10)

,words
197,nao
198,suas
199,seja
200,somos
201,nos
202,eram
203,houvessem
204,numa
205,de
206,estivemos


In [24]:
stop_words = df_stopwords['words'].tolist()

In [25]:
def clean_text(text, stop_words):
    text = [token for token in text.split(" ")]
    text = [word for word in text if not word in stop_words] # remove stopwords
    text = " ".join(text)
    return text

In [26]:
def cleaning(df,column, stop_words):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    # Remove stopwords
    df.loc[:,column] = df[column].apply(lambda x: clean_text(x, stop_words))

    return df

In [27]:
def remove_duplicates(sentence):
	words = sentence.split(" ")
	result = []
	for word in words:
		if word not in result:
			result.append(word)
	return result

<h3>Selecting the Departments</h3>

In [28]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-03-25 23:54:28.301 | INFO     | __main__:<module>:4 - boards_list: ['usi', 'cib', 'bb_asset', 'dimep', 'coger gesub', 'disem']


<h3>Parameters</h3>

In [29]:
from sklearn.metrics import make_scorer
import sklearn.metrics as sk

METRICS = {
           'accuracy': make_scorer(sk.accuracy_score),
           'precision': make_scorer(sk.precision_score, average = 'weighted', zero_division=0),
           'recall': make_scorer(sk.recall_score, average = 'weighted', zero_division=0),
           'f1': make_scorer(sk.f1_score, average = 'binary', zero_division=0),
           'f1_macro': make_scorer(sk.f1_score, average = 'macro', zero_division=0),
           'f1_weighted': make_scorer(sk.f1_score, average = 'weighted', zero_division=0)
           }

CROSS_VALIDATION = 10
RAND_STATE = 42
REFIT_VAL = 'f1'

cls = dict({
        'random_forest': {
            'estimator': RandomForestClassifier(random_state=RAND_STATE, n_jobs = NUM_PROCESSORS),
            'parameters':{'n_estimators': [100, 400, 1000], 'max_depth': [10,30, 100], 'criterion':['entropy', 'log_loss', 'gini']}
            },
        'svm': {
            'estimator': SVC(random_state=RAND_STATE),
            'parameters':{'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear','poly','rbf','sigmoid']}
            },
        'xgboost':{
            'estimator': XGBClassifier(random_state=RAND_STATE, num_class=2, verbosity = 0, silent=True, n_jobs=NUM_PROCESSORS),
            'parameters':{'objective':['reg:squarederror','binary:logistic','multi:softmax','binary:hinge'],'n_estimators': [100,1000],'max_depth': [10,30],'learning_rate':[0.01,0.5]}
        },
        'nb':{
                'estimator': MultinomialNB(),
                'parameters':{'alpha':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001],'force_alpha': [True, False],'fit_prior': [True, False]}
            }
        
    })

In [30]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def training_each_inductor_holdout(classifiers, board, X, y):
    for key in list(classifiers.keys()):
        lg.info(f"\n\n####### {key} INDUCTOR EXECUTING ####### - {board}\n\n")

        lg.debug(f"[+] classifiers[key]['estimator']: {classifiers[key]['estimator']} - {key} - {board}")
        lg.debug(f"[+] classifiers[key]['parameters']: {classifiers[key]['parameters']} - {key} - {board}")

        search = GridSearchCV(estimator=classifiers[key]['estimator'],
                            param_grid=classifiers[key]['parameters'],
                            cv = CROSS_VALIDATION,
                            scoring = METRICS,
                            refit = REFIT_VAL,
                            n_jobs = NUM_PROCESSORS,
                            verbose = VERBOSE)

        lg.debug(f"[+] Start GridSearchCV - {key} - {board}")
        search.fit(X, y)
        lg.debug(f"[+] End GridSearchCV - {key} - {board}")

        lg.info(f"search.best_estimator_: {search.best_estimator_} \n")

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=RAND_STATE, stratify=y)

        clf = search.best_estimator_.fit(x_train, y_train)
        
        y_pred = clf.predict(x_test)

        lg.success(f"\n\n####### METRICS {key} - {board} #######\n\n")

        lg.success(f"{key} - {board} - test_precision: {metrics.precision_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_recall: {metrics.recall_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_binary: {metrics.f1_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_macro: {metrics.f1_score(y_test, y_pred, average='macro')}")

<h3>The Training and Evaluation</h3>

In [ ]:
for board in boards_list:
    lg.info(f"\n\n####### {board} EXECUTING #######\n\n")

    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == board].shape[0]
    
    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # total_relevant_documents_of_the_board = int(data_relevant[data_relevant.department == board].shape[0]/10)

    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    lg.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {board}")
    lg.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == board].shape[0]}")

    data_relevant_board = data_relevant[data_relevant.department == board]
    lg.info(f"data_relevant_board: {data_relevant_board.shape} - {board}")

    ####################################################################################################
    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.

    data_irrelevant_board = data_irrelevant[data_irrelevant.department == board][:total_relevant_documents_of_the_board]
    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]

    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    ####################################################################################################

    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # data_relevant_board = data_relevant[data_relevant.department == board][:total_relevant_documents_of_the_board]
    
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]
    lg.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {board}")

    df_gather_datasets = []
    df_gather_datasets.append(data_relevant_board)
    df_gather_datasets.append(data_irrelevant_board)
    # Merging all dataframes in the list as one dataframe.
    df_final_dataset = pd.concat(df_gather_datasets)
    
    lg.info(f"df_final_dataset: {df_final_dataset.shape} - {board}")

    # Cleaning the text and title
    lg.debug("[+] Start cleaning text and title")
    df_final_dataset = remove_special_characters(df_final_dataset, 'text')
    df_final_dataset = remove_special_characters(df_final_dataset, 'subject')
    df_final_dataset = cleaning(df_final_dataset, 'text', stop_words)
    df_final_dataset = cleaning(df_final_dataset, 'subject', stop_words)
    lg.debug("[+] End cleaning text and title")

    # To get only phrases with at lest three words after the preprocessing.
    df_final_dataset = df_final_dataset[df_final_dataset['text'].str.len() >= 3]

    data_x = df_final_dataset.text
    lg.debug("[+] Start TF-IDF")
    
    feature_extraction = TfidfVectorizer()
    data_x = feature_extraction.fit_transform(data_x)
    lg.debug("[+] End TF-IDF")

    data_y = df_final_dataset['class']

    training_each_inductor_holdout(cls, board, data_x, data_y)

    lg.success("\n\n[+][+][+] SUCCESS! [+][+][+]\n\n")